In [1]:
import numpy as np
import pandas as pd

In [26]:
video_root = '/Users/inaki-eab/Desktop/small-fast-detector/tracker/evaluation/TrackEval/data/gt/mot_challenge/MOTHupba-train/MOT17-09'
gt_path = video_root + '/gt/gt.txt'
info_path = video_root + '/seqinfo.ini'

In [27]:
import configparser

config = configparser.ConfigParser()
# Read the .ini file
config.read(info_path)
frame_width = int(config['Sequence']['imWidth'])
frame_height = int(config['Sequence']['imHeight'])
video_fps = int(config['Sequence']['frameRate'])

In [28]:
df = pd.read_csv(gt_path, names= ['frame', 'id','xl', 'yt','w', 'h'], usecols=[0,1,2,3,4,5])

In [29]:
df

,frame,id,xl,yt,w,h
0,1,1,260,450,102,262
1,2,1,262,449,102,263
2,3,1,264,449,102,263
3,4,1,266,448,102,264
4,5,1,268,448,102,264
...,...,...,...,...,...,...
10406,207,64,881,531,36,86
10407,208,64,882,531,35,85
10408,209,64,883,532,35,84
10409,210,64,884,532,35,84


In [30]:
print('Num frames: ', df['frame'].max())
print('Num IDs: ', df['id'].unique().max())

Num frames:  525
Num IDs:  64


In [31]:
# Compute Bounding Box properties
df['area'] = df['w']*df['h']
df['xc'] = df['xl'] + df['w']/2
df['yc'] = df['yt'] + df['h']/2

In [32]:
df

,frame,id,xl,yt,w,h,area,xc,yc
0,1,1,260,450,102,262,26724,311.0,581.0
1,2,1,262,449,102,263,26826,313.0,580.5
2,3,1,264,449,102,263,26826,315.0,580.5
3,4,1,266,448,102,264,26928,317.0,580.0
4,5,1,268,448,102,264,26928,319.0,580.0
...,...,...,...,...,...,...,...,...,...
10406,207,64,881,531,36,86,3096,899.0,574.0
10407,208,64,882,531,35,85,2975,899.5,573.5
10408,209,64,883,532,35,84,2940,900.5,574.0
10409,210,64,884,532,35,84,2940,901.5,574.0


In [33]:
interest_point = np.array([frame_width//2, frame_height])
trigger_radius = frame_height/4 

# Min distance to interest point
def minimal_distance_to_bbox(P, bbox):
    x_center, y_center, w, h = bbox
    
    x_min = x_center - w / 2
    x_max = x_center + w / 2
    y_min = y_center - h / 2
    y_max = y_center + h / 2
    
    # Closest point on the bounding box to P
    closest_x = max(x_min, min(P[0], x_max))
    closest_y = max(y_min, min(P[1], y_max))
    
    # Compute the Euclidean distance between P and the closest point
    distance = np.sqrt((P[0] - closest_x) ** 2 + (P[1] - closest_y) ** 2)
    
    return distance

# Function to apply on each row of the DataFrame
def near_sensor(row):
    bbox = (row['xc'], row['yc'], row['w'], row['h'])
    d = minimal_distance_to_bbox(interest_point, bbox)
    return True if d <= trigger_radius else False

# Apply the function to each row and create a new column
df['near_sensor'] = df.apply(near_sensor, axis=1)

In [34]:
df[df['near_sensor']==True]

,frame,id,xl,yt,w,h,area,xc,yc,near_sensor
654,386,2,1007,337,189,478,90342,1101.5,576.0,True
655,387,2,1001,337,192,480,92160,1097.0,577.0,True
656,388,2,996,337,195,481,93795,1093.5,577.5,True
657,389,2,990,337,198,483,95634,1089.0,578.5,True
658,390,2,985,338,201,484,97284,1085.5,580.0,True
...,...,...,...,...,...,...,...,...,...,...
7203,243,29,961,290,214,544,116416,1068.0,562.0,True
7204,244,29,975,289,203,540,109620,1076.5,559.0,True
7205,245,29,988,288,193,536,103448,1084.5,556.0,True
7206,246,29,1002,287,182,532,96824,1093.0,553.0,True


In [35]:
from itertools import combinations
import networkx as nx

def recognize_gather(df, frame_id, area_threshold, distance_threshold, min_people):
    
    frame_df = df[df['frame']==frame_id].copy()
    frame_df.reset_index(inplace=True, drop=True)
    
    pairs = []
    for i,j in combinations(range(len(frame_df)), 2):
        # Check similarity of areas
        # TODO: order numerator and denominator
        if area_threshold <= (frame_df.loc[i, 'area']/frame_df.loc[j, 'area']) <= (1/area_threshold):
            # Compute euclidean distance
            d = np.sqrt(np.sum((frame_df.loc[i, ['xc','yc']] - frame_df.loc[j, ['xc','yc']])**2))
            # Get mean area
            a = (frame_df.loc[i, 'area'] + frame_df.loc[j, 'area'])
            # Normalize distance
            norm_d = d/np.sqrt(a)
            # Check distance ¿speed?
            if norm_d <= distance_threshold:
                pairs.append([i,j])
    
    # Get independent chains
    g = nx.Graph()
    g.add_edges_from(pairs)
    # Find connected components
    independent_chains = list(nx.connected_components(g))
    # Filter out chains having less than 3 elements
    valid_chains = [chain for chain in independent_chains if len(chain) > (min_people-1)]   
    
    # Assing group tags to the corresponding bboxes
    for i, chain in enumerate(valid_chains):
        frame_df.loc[list(chain), 'G'] = i+1

    new_df = pd.merge(df, frame_df[['frame', 'id', 'G']], on=['frame', 'id'], how='left')
    return new_df

In [70]:
area_threshold = 0.5
distance_threshold = 1
min_people = 3

frame_id = 3

new_df = recognize_gather(df, frame_id, area_threshold, distance_threshold, min_people)
new_df

,frame,id,xl,yt,w,h,area,xc,yc,near_sensor,G
0,1,1,260,450,102,262,26724,311.0,581.0,False,NaN
1,2,1,262,449,102,263,26826,313.0,580.5,False,NaN
2,3,1,264,449,102,263,26826,315.0,580.5,False,1.0
3,4,1,266,448,102,264,26928,317.0,580.0,False,NaN
4,5,1,268,448,102,264,26928,319.0,580.0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
10406,207,64,881,531,36,86,3096,899.0,574.0,False,NaN
10407,208,64,882,531,35,85,2975,899.5,573.5,False,NaN
10408,209,64,883,532,35,84,2940,900.5,574.0,False,NaN
10409,210,64,884,532,35,84,2940,901.5,574.0,False,NaN


In [71]:
new_df[new_df['frame']==frame_id]

,frame,id,xl,yt,w,h,area,xc,yc,near_sensor,G
2,3,1,264,449,102,263,26826,315.0,580.5,False,1.0
3545,3,19,1684,386,170,348,59160,1769.0,560.0,False,NaN
3745,3,20,1881,325,157,408,64056,1959.5,529.0,False,NaN
3965,3,21,1259,535,61,125,7625,1289.5,597.5,False,NaN
4347,3,22,1295,457,72,204,14688,1331.0,559.0,False,NaN
4739,3,23,-331,234,469,694,325486,-96.5,581.0,False,NaN
5043,3,25,1035,174,136,532,72352,1103.0,440.0,False,NaN
5568,3,26,115,521,84,230,19320,157.0,636.0,False,1.0
6093,3,27,234,395,21,440,9240,244.5,615.0,False,NaN
6618,3,28,1682,470,64,121,7744,1714.0,530.5,False,NaN


In [38]:
def get_motion_descriptors(df, id, w, dt, l):

    # Select ID, should be paralelized using group by
    id_df = df[df['id']==id].copy()
    
    # Compute projected instant diferentials
    id_df.loc[:,'dx'] = (id_df['xc'] - id_df['xc'].shift(1)) * w[0]
    id_df.loc[:,'dy'] = (id_df['yc'] - id_df['yc'].shift(1)) * w[1]
    
    # Compute area-normalized projected instant speed
    id_df.loc[:,'dv'] = np.sqrt(id_df['dx']**2 + id_df['dy']**2) / (id_df['area'])
    
    # Compute interval projected differences
    id_df.loc[:,'Dx'] = (id_df['xc'] - id_df['xc'].shift(dt)) * w[0]
    id_df.loc[:,'Dy'] = (id_df['yc'] - id_df['yc'].shift(dt)) * w[1]
    
    # Compute area-normalized projected speed
    id_df.loc[:,'V'] = np.sqrt(id_df['Dx']**2 + id_df['Dy']**2) / (dt + id_df['area'])
    
    # Average the speed according to last L*dt observations
    id_df.loc[:,'aV'] = id_df['V'].rolling(window=dt*l, min_periods=1).mean()
    
    # Compute average and instant direction of movement # TODO: which one should we use?
    id_df.loc[:, 'idir'] = np.sign(id_df['dy'])
    id_df.loc[:, 'aidir'] = id_df['idir'].rolling(window=dt*l, min_periods=1).mean()
    id_df.loc[:, 'dir'] = np.sign(id_df['Dy'])
    id_df.loc[:, 'adir'] = id_df['dir'].rolling(window=dt*l, min_periods=1).mean()

    new_df = pd.merge(df, id_df[['frame', 'id', 'dv', 'aV', 'aidir', 'adir']], on=['frame', 'id'], how='left')
    return new_df

In [39]:
# Constants
w = [0.7, 1.5] #projection
dt = 30 #window frames
l = 5 #list's length

id = 31

new_df = get_motion_descriptors(new_df, id, w, dt, l)

In [40]:
new_df[new_df['id']==id]

,frame,id,xl,yt,w,h,area,xc,yc,near_sensor,G,dv,aV,aidir,adir
7930,1,31,863,522,46,108,4968,886.0,576.0,False,NaN,NaN,NaN,NaN,NaN
7931,2,31,863,521,46,108,4968,886.0,575.0,False,NaN,0.000302,NaN,-1.000000,NaN
7932,3,31,864,521,46,108,4968,887.0,575.0,False,NaN,0.000141,NaN,-0.500000,NaN
7933,4,31,865,521,46,108,4968,888.0,575.0,False,NaN,0.000141,NaN,-0.333333,NaN
7934,5,31,865,521,46,108,4968,888.0,575.0,False,NaN,0.000000,NaN,-0.250000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8385,456,31,773,516,61,126,7686,803.5,579.0,False,NaN,0.000215,0.002930,0.073333,0.033333
8386,457,31,771,516,61,126,7686,801.5,579.0,False,NaN,0.000182,0.002948,0.073333,0.033333
8387,458,31,769,516,61,126,7686,799.5,579.0,False,NaN,0.000182,0.002966,0.073333,0.033333
8388,459,31,768,517,61,126,7686,798.5,580.0,False,NaN,0.000215,0.002984,0.080000,0.033333


In [41]:
new_df['SS'] = new_df['aV'] < 0.001
new_df[new_df['SS']==True]

,frame,id,xl,yt,w,h,area,xc,yc,near_sensor,G,dv,aV,aidir,adir,SS
8078,149,31,899,517,49,116,5684,923.5,575.0,False,NaN,0.000000,0.000997,0.000000,0.109244,True
8079,150,31,899,517,49,116,5684,923.5,575.0,False,NaN,0.000000,0.000991,0.000000,0.116667,True
8080,151,31,900,517,48,116,5568,924.0,575.0,False,NaN,0.000063,0.000984,0.000000,0.123967,True
8081,152,31,900,517,48,116,5568,924.0,575.0,False,NaN,0.000000,0.000977,0.006667,0.131148,True
8082,153,31,900,517,48,116,5568,924.0,575.0,False,NaN,0.000000,0.000971,0.006667,0.138211,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8275,346,31,928,512,49,122,5978,952.5,573.0,False,NaN,0.000000,0.000990,0.013333,-0.486667,True
8276,347,31,928,512,49,122,5978,952.5,573.0,False,NaN,0.000000,0.000991,0.013333,-0.480000,True
8277,348,31,928,512,49,122,5978,952.5,573.0,False,NaN,0.000000,0.000992,0.013333,-0.473333,True
8278,349,31,928,512,49,122,5978,952.5,573.0,False,NaN,0.000000,0.000995,0.013333,-0.466667,True


In [42]:
new_df

,frame,id,xl,yt,w,h,area,xc,yc,near_sensor,G,dv,aV,aidir,adir,SS
0,1,1,260,450,102,262,26724,311.0,581.0,False,1.0,NaN,NaN,NaN,NaN,False
1,2,1,262,449,102,263,26826,313.0,580.5,False,NaN,NaN,NaN,NaN,NaN,False
2,3,1,264,449,102,263,26826,315.0,580.5,False,NaN,NaN,NaN,NaN,NaN,False
3,4,1,266,448,102,264,26928,317.0,580.0,False,NaN,NaN,NaN,NaN,NaN,False
4,5,1,268,448,102,264,26928,319.0,580.0,False,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10406,207,64,881,531,36,86,3096,899.0,574.0,False,NaN,NaN,NaN,NaN,NaN,False
10407,208,64,882,531,35,85,2975,899.5,573.5,False,NaN,NaN,NaN,NaN,NaN,False
10408,209,64,883,532,35,84,2940,900.5,574.0,False,NaN,NaN,NaN,NaN,NaN,False
10409,210,64,884,532,35,84,2940,901.5,574.0,False,NaN,NaN,NaN,NaN,NaN,False


In [62]:
def get_intervals(df, flag):

    df = new_df.copy()
    # Sort the DataFrame by object_id and frame
    df = df.sort_values(by=['id', 'frame'])
    
    # Create a group identifier that changes when the flag changes
    df['group'] = (df[flag] != df.groupby('id')[flag].shift()).cumsum()
    
    # Aggregate to get the start and end frames for each interval
    intervals = df.groupby(['id', flag, 'group']).agg(start_frame=('frame', 'min'),
                                                               end_frame=('frame', 'max')).reset_index()
    
    # Drop the group column as it is no longer needed
    intervals = intervals.drop(columns=['group'])
    
    # Only positive intervals 
    positive = intervals[intervals[flag]==True]

    return positive

In [63]:
flag='SS'
ss_df = get_intervals(new_df, flag)
ss_df

,id,SS,start_frame,end_frame
32,31,True,149,350


In [45]:
new_df['SR'] = new_df['dv'] > 0.025
get_intervals(new_df, 'SR')

,id,SR,start_frame,end_frame


In [46]:
new_df['FA'] = (new_df['aidir'] > 0) & new_df['near_sensor']
get_intervals(new_df, 'FA')

,id,FA,start_frame,end_frame


In [49]:
g_df = get_intervals(new_df, 'G')

In [50]:
ss_df

,id,SS,start_frame,end_frame
32,31,True,149,350


In [64]:
g_df

,id,G,start_frame,end_frame
0,1,1.0,1,1
1,26,1.0,1,1
2,30,1.0,1,1


In [74]:
pd.concat([ss_df, g_df])

,id,SS,start_frame,end_frame,G
32,31,True,149,350,NaN
0,1,NaN,1,1,1.0
1,26,NaN,1,1,1.0
2,30,NaN,1,1,1.0
